# Manipulating DataFrames in the real world

A look at various techniques to modify the contents of DataFrames in Spark.

## Filtering column content with Python
You've looked at using various operations on DataFrame columns - now you can modify a real dataset. The DataFrame voter_df contains information regarding the voters on the Dallas City Council from the past few years. This truncated DataFrame contains the date of the vote being cast and the name and position of the voter. Your manager has asked you to clean this data so it can later be integrated into some desired reports. The primary task is to remove any null entries or odd characters and return a specific set of voters where you can validate their information.

This is often one of the first steps in data cleaning - removing anything that is obviously outside the format. For this dataset, make sure to look at the original data and see what looks out of place for the VOTER_NAME column.

In [2]:
import pyspark.sql.functions as F

# File Path
file_path = ".../data/datacamp/voter/"

voter_df = spark.read.parquet(file_path)

# Show the distinct VOTER_NAME entries
voter_df.select("VOTER_NAME").distinct().show(5, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains("_"))

# Show the distinct VOTER_NAME entries again
voter_df.select("VOTER_NAME").distinct().show(5, truncate=False)

+-----------------+
|VOTER_NAME       |
+-----------------+
|null             |
|011018__42       |
|the final 201... |
|Tiffinni A. Young|
|Monica R. Alonzo |
+-----------------+
only showing top 5 rows

+-----------------+
|VOTER_NAME       |
+-----------------+
|the final 201... |
|Tiffinni A. Young|
|Monica R. Alonzo |
|Kevin Felder     |
|Mark Clayton     |
+-----------------+
only showing top 5 rows



## Modifying DataFrame columns
Previously, you filtered out any rows that didn't conform to something generally resembling a name. Now based on your earlier work, your manager has asked you to create two new columns - first_name and last_name. She asks you to split the VOTER_NAME column into words on any space character. You'll treat the last word as the last_name, and all other words as the first_name. You'll be using some new functions in this exercise including .split(), .size(), and .getItem(). The .getItem(index) takes an integer value to return the appropriately numbered item in the column. The functions .split() and .size() are in the pyspark.sql.functions library.

Please note that these operations are always somewhat specific to the use case. Having your data conform to a format often matters more than the specific details of the format. Rarely is a data cleaning task meant just for one person - matching a defined format allows for easier sharing of the data later (ie, Paul doesn't need to worry about names - Mary already cleaned the dataset).

In [3]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show(5)

+----------+-------------+---------------+----------+---------+
|      DATE|        TITLE|     VOTER_NAME|first_name|last_name|
+----------+-------------+---------------+----------+---------+
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|
+----------+-------------+---------------+----------+---------+
only showing top 5 rows



## when() example
The when() clause lets you conditionally modify a Data Frame based on its content. You'll want to modify our voter_df DataFrame to add a random number to any voting member that is defined as a "Councilmember".

In [4]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == "Councilmember", F.rand()))

# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show(5)

+----------+-------------+---------------+----------+---------+-------------------+
|      DATE|        TITLE|     VOTER_NAME|first_name|last_name|         random_val|
+----------+-------------+---------------+----------+---------+-------------------+
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough| 0.5088622853656986|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.22711379138967347|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough| 0.7076059242271496|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough| 0.9501812886138689|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough| 0.9778281065662836|
+----------+-------------+---------------+----------+---------+-------------------+
only showing top 5 rows



## When / Otherwise
This requirement is similar to the last, but now you want to add multiple values based on the voter's position. Modify your voter_df DataFrame to add a random number to any voting member that is defined as a Councilmember. Use 2 for the Mayor and 0 for anything other position.

In [5]:
from pyspark.sql.functions import when

# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               when(voter_df.TITLE == 'Councilmember', F.rand()) \
                               .when(voter_df.TITLE == "Mayor", 2) \
                               .otherwise(0))

# Show some of the DataFrame rows
voter_df.show(5)

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val == 0).show(5)

+----------+-------------+---------------+----------+---------+-------------------+
|      DATE|        TITLE|     VOTER_NAME|first_name|last_name|         random_val|
+----------+-------------+---------------+----------+---------+-------------------+
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|  0.740034089413097|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.18192571641220723|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough| 0.7778368240852591|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.49766880191751284|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.28931653273216695|
+----------+-------------+---------------+----------+---------+-------------------+
only showing top 5 rows

+----------+-------------+-----------------+----------+---------+----------+
|      DATE|        TITLE|       VOTER_NAME|first_name|last_name|random_val|
+----------+-------------+-----------------+----------+---------+

## Using user defined functions in Spark
You've seen some of the power behind Spark's built-in string functions when it comes to manipulating DataFrames. However, once you reach a certain point, it becomes difficult to process the data in a without creating a rat's nest of function calls. Here's one place where you can use User Defined Functions to manipulate our DataFrames.

For this exercise, we'll use our voter_df DataFrame, but you're going to replace the first_name column with the first and middle names.

In [6]:
from pyspark.sql.types import *

def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[:-1])

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+')) 
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Show the DataFrame
voter_df.show(5)

+----------+-------------+---------------+----------+---------+-------------------+-------------------+---------------------+
|      DATE|        TITLE|     VOTER_NAME|first_name|last_name|         random_val|             splits|first_and_middle_name|
+----------+-------------+---------------+----------+---------+-------------------+-------------------+---------------------+
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|  0.740034089413097|[B., Adam, McGough]|              B. Adam|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.18192571641220723|[B., Adam, McGough]|              B. Adam|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough| 0.7778368240852591|[B., Adam, McGough]|              B. Adam|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.49766880191751284|[B., Adam, McGough]|              B. Adam|
|09/28/2016|Councilmember|B. Adam McGough|        B.|  McGough|0.28931653273216695|[B., Adam, McGough]|              B

## Adding an ID Field
When working with data, you sometimes only want to access certain fields and perform various operations. In this case, find all the unique voter names from the DataFrame and add a unique ID number. Remember that Spark IDs are assigned based on the DataFrame partition - as such the ID values may be much greater than the actual number of rows in the DataFrame.

With Spark's lazy processing, the IDs are not actually generated until an action is performed and can be somewhat random depending on the size of the dataset.

In [7]:
# File Path
file_path = ".../data/datacamp/votes"

# Load in data
df = spark.read.parquet(file_path)

# Select all the unique council voters
voter_df = df.select(df["VOTERNAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(5)


There are 33 rows in the voter_df DataFrame.

+-------------------+--------------+
|          VOTERNAME|        ROW_ID|
+-------------------+--------------+
|    Lee M. Kleinman|40742059769856|
| Rickey D. Callahan|40638980554752|
| Philip T. Kingston|38920993636352|
|      Casey  Thomas|38147899523072|
|Carolyn King Arnold|37709812858880|
+-------------------+--------------+
only showing top 5 rows



In [8]:
# Repartition
voter_df_single = voter_df.repartition(1)

# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())
print("\nThere are %d partitions in the voter_df_single DataFrame.\n" % voter_df_single.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_single = voter_df_single.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df.ROW_ID.desc()).show(5)
voter_df_single.orderBy(voter_df_single.ROW_ID.desc()).show(5)


There are 5000 partitions in the voter_df DataFrame.


There are 1 partitions in the voter_df_single DataFrame.

+-------------------+--------------+
|          VOTERNAME|        ROW_ID|
+-------------------+--------------+
|    Lee M. Kleinman|40742059769856|
| Rickey D. Callahan|40638980554752|
| Philip T. Kingston|38920993636352|
|      Casey  Thomas|38147899523072|
|Carolyn King Arnold|37709812858880|
+-------------------+--------------+
only showing top 5 rows

+-------------------+------+
|          VOTERNAME|ROW_ID|
+-------------------+------+
|    Lee M. Kleinman|    32|
| the  final   20...|    31|
|Rickey D.  Callahan|    30|
|      Casey  Thomas|    29|
|       Casey Thomas|    28|
+-------------------+------+
only showing top 5 rows



Notice the drastic difference in the 'ROW_ID' values between the two Data Frames. Understanding how lazy processing and partitioning behave are integral to mastering Spark. Make sure to always test your assumptions when creating a Spark workflow to avoid nasty suprises in production.

## More ID tricks
Once you define a Spark process, you'll likely want to use it many times. Depending on your needs, you may want to start your IDs at a certain value so there isn't overlap with previous runs of the Spark task. This behavior is similar to how IDs would behave in a relational database. You have been given the task to make sure that the IDs output from a monthly Spark task start at the highest value from the previous month.

In [9]:
# File Path
file_path = ".../data/datacamp/voter_month/"

# Load in data
voter_df_march = spark.read.csv(file_path + 'voter_march.csv', header=True)
voter_df_april = spark.read.csv(file_path + 'voter_april.csv', header=True)

# Determine the highest ROW_ID and save it in previous_max_ID
previous_max_ID = voter_df_march.select('ROW_ID').rdd.max()[0]

# Add a ROW_ID column to voter_df_april starting at the desired value
voter_df_april = voter_df_april.withColumn('ROW_ID', F.monotonically_increasing_id() + previous_max_ID)

# Show the ROW_ID from both DataFrames and compare
voter_df_march.select('ROW_ID').show(5)
voter_df_april.select('ROW_ID').show(5)

+------------+
|      ROW_ID|
+------------+
|  8589934592|
| 34359738368|
| 42949672960|
| 51539607552|
|103079215104|
+------------+
only showing top 5 rows

+----------------+
|          ROW_ID|
+----------------+
|9.62072674304E11|
|9.62072674305E11|
|9.62072674306E11|
|9.62072674307E11|
|9.62072674308E11|
+----------------+
only showing top 5 rows



Fantastic work! It's easy to forget that the output of a Spark method can often be modified before being assigned. This provides a lot of power and flexibility, especially when trying to migrate tasks from various technologies. Consider how you could use everything we've learned in this chapter to create a combination ID containing a name, a new ID, and perhaps a conditional value. When you are able to view your tasks as compositions of available functions, you can clean and modify your data in any way you see fit.